In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.compose import ColumnTransformer

In [3]:
df=pd.read_csv('G:\Campus x\dataset\heart_disease_data.csv')
df.head(3)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [5]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [6]:
df['target'].value_counts()

1    165
0    138
Name: target, dtype: int64

In [7]:
df.groupby('target').mean()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
target,,,,,,,,,,,,,
0,56.601449,0.826087,0.478261,134.398551,251.086957,0.159420,0.449275,139.101449,0.550725,1.585507,1.166667,1.166667,2.543478
1,52.496970,0.563636,1.375758,129.303030,242.230303,0.139394,0.593939,158.466667,0.139394,0.583030,1.593939,0.363636,2.121212


# Separate Dependent and Independent Variables

In [8]:
x=df.drop(columns=['target'])
y=df['target']

In [9]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)

In [30]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model1',LogisticRegression(C=0.1, max_iter=100.0, penalty='l2', solver='liblinear'))
])

In [31]:
from sklearn import set_config
set_config(display='diagram')

In [32]:
pipe.fit(xtrain,ytrain)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model1',
                 LogisticRegression(C=0.1, max_iter=100.0,
                                    solver='liblinear'))])

In [33]:
ypred=pipe.predict(xtest)

In [34]:
accuracy_score(ypred,ytest)

0.8688524590163934

# cross val score

In [20]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, xtrain, ytrain, cv=3, scoring='accuracy').mean()

0.8018004115226337

In [21]:
from sklearn.model_selection import GridSearchCV

In [22]:
param_grid = {'C': [0.1, 1, 10], 
              'penalty': ['l1', 'l2'], 
              'solver': ['liblinear', 'saga'], 
              'max_iter': np.linspace(100,500,10)}

In [25]:
grid = GridSearchCV(estimator=LogisticRegression(), param_grid=param_grid, cv=5, n_jobs=-1)

In [27]:
grid.fit(x, y)

GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10],
                         'max_iter': array([100.        , 144.44444444, 188.88888889, 233.33333333,
       277.77777778, 322.22222222, 366.66666667, 411.11111111,
       455.55555556, 500.        ]),
                         'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']})

In [29]:
print("Best parameters: ", grid.best_params_)
print("Best accuracy score: ", grid.best_score_)

Best parameters:  {'C': 0.1, 'max_iter': 100.0, 'penalty': 'l2', 'solver': 'liblinear'}
Best accuracy score:  0.834808743169399
